# ESTE ES EL NOTEBOOK DEL TFM

## Importar librerías

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import os
import re

import missingno as msno
import sys
from fuzzywuzzy import process

import warnings
warnings.filterwarnings('ignore')

## Cargar datos

In [3]:
ruta = r'C:\Users\JAIME\Documents\GitHub\TFM-Sesgos-en-el-sistema-judicial-de-EEUU-\00_Data\00_Raw'

lista_ficheros = os.listdir(ruta)

lista_ficheros_tfm = [fichero for fichero in lista_ficheros if '.csv' in fichero]

In [15]:
dict_datos = {}

for fichero in lista_ficheros_tfm:
    ruta_completa = os.path.join(ruta, fichero)
    df = pd.read_csv(ruta_completa)
    clave = os.path.splitext(fichero.replace('-', '_'))[0]
    dict_datos[clave] = df

dict_datos.keys()

dict_keys(['compas_scores_raw', 'compas_scores_two_years'])

## Exploración Inicial

In [1]:
print('hola mundo')

hola mundo


In [ ]:
print("me corroooo")